<H3> Autor: </H3> Pablo Meana González

<h2><font color="blue" >CREACION BASE DE DATOS ESTUDIO</font></h2><p>

In [44]:
# Librerias necesarias
import openpyxl
import pandas as pd
import numpy as np
import folium
from folium import plugins
import webbrowser
from pyproj import Transformer

In [91]:
# Creamos la Base de datos
base_datos= pd.read_excel('bbdd_mediciones.xlsx')
# Cambiar aquí el año a 2023, lo va a pedir para la entrega
# Descomentar esto
# base_datos_1= pd.read_excel('2023_Accidentalidad.xlsx')
base_datos.head(5)
# base_datos_1.head(5)

,id_medicion,tipo_medicion,latitud,longitud
0,1,O3,39.0180,-0.17167
1,2,NO2,39.0164,-0.17104
2,3,SO3,39.0172,-0.17992
3,4,TEMPERATURA,39.0158,-0.17188
4,5,O3,39.0153,-0.17821


# Preparación Base de datos

In [92]:
print(base_datos.dtypes) # imprime el tipo de campo
# print(base_datos_1.dtypes)
print("Numero de registros =" , len( base_datos)) # imprime el tamaño de la base de datos
# print("Numero de registros =" , len( base_datos_1)) # imprime el tamaño de la base de datos

id_medicion        int64
tipo_medicion     object
latitud          float64
longitud         float64
dtype: object
Numero de registros = 6070


In [93]:
mediciones_df = base_datos.dropna(subset=['latitud', 'longitud'])

<h2><font color="blue" >Mapa de Calor (HeatMap)</font></h2><p>

Los mapas de calor, o heatmaps, son un tipo de representación dentro de de la simbología de mapas que nos permite mostrar al usuario los puntos calientes o conjuntos de datos más relevantes dentro de una nube de puntos. Con una combinación de opciones de simbología y datos cuantitativos provenientes de base de datos georeferenciadas, obtendras mapas que reflejan concentraciones o distribuciones espaciales de variables no continuas en el espacio.

In [100]:
# Utilizaremos el pluging HeatMap (mapa de calor se llama Heatmap)
# la lista de datos puede ser de 2 formas (Latitud, Longitud) o (Latitud, Longitud y Peso).
# Mapa de calor de accidentes de Madrid
# 2 formas: que tenga en cuenta los pesos y que no tenga en cuenta los pesos. Si hay 20 accidentes, le da un x20 de valor
mapa_calor = folium.Map(location=(39.007541,-0.167470), tiles = 'cartodb positron', zoom_start = 13)

O3_df=mediciones_df.loc[(mediciones_df['tipo_medicion']=='O3')] 
SO3_df=mediciones_df.loc[(mediciones_df['tipo_medicion']=='NO2')]
NO2_df=mediciones_df.loc[(mediciones_df['tipo_medicion']=='SO3')]
TEMPERATURA_df=mediciones_df.loc[(mediciones_df['tipo_medicion']=='TEMPERATURA')]
# Crear el mapa de calor
plugins.HeatMap(data=O3_df[['latitud', 'longitud']], radius=15, use_local_extrema=True, name="Mapa de O3").add_to(mapa_calor)
plugins.HeatMap(data=SO3_df[['latitud', 'longitud']], radius=15, use_local_extrema=True, name="Mapa de SO3").add_to(mapa_calor)
plugins.HeatMap(data=NO2_df[['latitud', 'longitud']], radius=15, use_local_extrema=True, name="Mapa de NO2").add_to(mapa_calor)
plugins.HeatMap(data=TEMPERATURA_df[['latitud', 'longitud']], radius=15, use_local_extrema=True, name="Mapa de Temperatura").add_to(mapa_calor)

folium.LayerControl().add_to(mapa_calor)
mapa_calor

In [101]:
# publicar el mapa
mapa_calor.save("heatmap_acci.html")
webbrowser.open_new_tab('heatmap_acci.html') #Abre el HTML en una ventana del navegador

True

 <h2><font color="blue" >Mapa de calor con evolución en el tiempo (HeatMapWithTime) </font></h2><p>
Este plugin nos permite generar mapas de calor de diferentes periodos de tiempo especificados y representarlos mediante automatizacion con un play que nos permite represesentar la evolución del fenómeno

In [ ]:
# Crear una nueva Dataframe que cumpla unas determinadas condiciones
acci_df=mediciones_df.loc[(mediciones_df['tipo_persona']=='Conductor') & (mediciones_df['tipo_vehiculo']=='Turismo')]

In [20]:
# Crear un campo en el Dataframe con el valor del mes extraido de la fecha
# aqui se saca el mes
import datetime
mes = pd.DatetimeIndex(acci_df['fecha']).month 
acci_df = acci_df.assign(mes = mes)  
acci_df.head()

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,...,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga,longitud,latitud,mes
0,2022S000001,2022-01-01,01:30:00,"AVDA. ALBUFERA, 19",19,13,PUENTE DE VALLECAS,Alcance,Despejado,Turismo,...,Mujer,NaN,NaN,443359.226,4472082.272,N,NaN,-3.667437,40.397420,1
1,2022S000001,2022-01-01,01:30:00,"AVDA. ALBUFERA, 19",19,13,PUENTE DE VALLECAS,Alcance,Despejado,Turismo,...,Hombre,NaN,NaN,443359.226,4472082.272,N,NaN,-3.667437,40.397420,1
4,2022S000002,2022-01-01,00:30:00,PLAZA. CANOVAS DEL CASTILLO / PASEO. PRADO,2,3,RETIRO,Colisión fronto-lateral,NaN,Turismo,...,Hombre,NaN,NaN,441155.351,4474129.588,N,NaN,-3.693594,40.415710,1
7,2022S000004,2022-01-01,03:40:00,CALL. ALCALA / PLAZA. EISENHOWER,728,20,SAN BLAS-CANILLEJAS,Choque contra obstáculo fijo,Despejado,Turismo,...,Hombre,2.0,Ingreso inferior o igual a 24 horas,449693.925,4477837.552,N,NaN,-3.593250,40.449674,1
9,2022S000005,2022-01-01,04:30:00,"AUTOV. A-42, +00500E",+00500E,12,USERA,Choque contra obstáculo fijo,Despejado,Turismo,...,Hombre,NaN,NaN,438952.303,4469985.021,S,NaN,-3.719162,40.378215,1


In [21]:
# Generar lista para el slider del visualizador
lista_tiempo = ['Enero','Febrero','Marzo','Abril','Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre','Diciembre']

In [22]:
lista_peso = [] # Crear una lista vacia
acci_df = acci_df.assign(peso = 1)  # crear un campo nuevo con valor por defecto 1

# Rellenar la lista con los campos (Latitud, Longitud y Peso) necesarios para HeatMap
for x in acci_df['mes'].sort_values().unique():
   lista_peso.append(acci_df.loc[acci_df['mes']==x,   # Agrupar los valores con latitud y longitud igual 
                  ['latitud','longitud','peso']].groupby(['latitud','longitud']).sum().reset_index().values.tolist())

In [23]:
print(lista_peso     )

[[[40.3313997866899, -3.714751009326987, 1.0], [40.33186454825793, -3.710183684853794, 1.0], [40.33397557600364, -3.7125745092758753, 1.0], [40.33498129284692, -3.695279764378348, 2.0], [40.33768324824523, -3.715403699281172, 2.0], [40.338102283226846, -3.709810167249936, 1.0], [40.33959358355342, -3.7064025884467644, 1.0], [40.34089340345841, -3.712143234568731, 2.0], [40.340974626121955, -3.704980348040892, 1.0], [40.341261708290354, -3.692724282838537, 4.0], [40.341649189684276, -3.6928405455813835, 1.0], [40.34357308268757, -3.702026911172522, 1.0], [40.3437698927299, -3.714769044316205, 2.0], [40.34414654503222, -3.7151906074403986, 1.0], [40.344248841875235, -3.701918469617324, 3.0], [40.34443139678639, -3.6929430602545175, 1.0], [40.344686266163095, -3.6936899756695865, 2.0], [40.34486026576901, -3.707463629506704, 2.0], [40.34492103167742, -3.7125549198009757, 1.0], [40.34496484715284, -3.6972260486131976, 2.0], [40.34511818560774, -3.691753600933707, 2.0], [40.345306886624535,

In [24]:
# Generar el mapa
madrid_map = folium.Map(location=(40.43,-3.65), control_scale = True, tiles = "cartodb positron", zoom_start = 12)

# Ejecutar el pluging HeatMapWithTime
plugins.HeatMapWithTime(lista_peso, radius = 30, index = lista_tiempo,
                        auto_play = False, min_opacity = 0.5, max_opacity = 1, use_local_extrema=True, name="Accidentes_Turismo_2022").add_to(madrid_map)

# publica el mapa
folium.LayerControl().add_to(madrid_map)
madrid_map.save("madrid_acci.html")
webbrowser.open_new_tab('madrid_acci.html')

True